In [6]:
### Finish the Solution Parse Part of Edit Distance
from functools import lru_cache
def edit_distance(string1: str, string2:str) -> int:
    solutions = {}
    @lru_cache(maxsize=2**10)
    def helper(string1: str, string2:str) -> int:
        if len(string1) == 0: 
            solutions[(string1, string2)] = 'ADD ' + string2
            return len(string2)
        if len(string2) == 0: 
            solutions[(string1, string2)] = 'DEL ' + string1
            return len(string1)

        tail_s1 = string1[-1]
        tail_s2 = string2[-1]

        candidates = [
            (helper(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)), # 删除 s1 的尾字母
            (helper(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)), # 添加 s2 的尾字母到 s1 的尾部
        ]

        if tail_s1 == tail_s2:
            both_forward = (helper(string1[:-1], string2[:-1]), '')
        else:
            both_forward = (helper(string1[:-1], string2[:-1])+1, 'SUB {} => {}'.format(tail_s1, tail_s2))

        candidates.append(both_forward)

        min_distance, operation = min(candidates)

        solutions[(string1, string2)] = operation

        return min_distance
    return helper(string1, string2), solutions

In [7]:
string1 = 'ago'
string2 = 'again'
distance, solutions = edit_distance(string1, string2)

In [8]:
solutions

{('', 'again'): 'ADD again',
 ('', 'agai'): 'ADD agai',
 ('', 'aga'): 'ADD aga',
 ('', 'ag'): 'ADD ag',
 ('', 'a'): 'ADD a',
 ('a', ''): 'DEL a',
 ('', ''): 'ADD ',
 ('a', 'a'): '',
 ('a', 'ag'): 'ADD g',
 ('a', 'aga'): '',
 ('a', 'agai'): 'ADD i',
 ('a', 'again'): 'ADD n',
 ('ag', ''): 'DEL ag',
 ('ag', 'a'): 'DEL g',
 ('ag', 'ag'): '',
 ('ag', 'aga'): 'ADD a',
 ('ag', 'agai'): 'ADD i',
 ('ag', 'again'): 'ADD n',
 ('ago', ''): 'DEL ago',
 ('ago', 'a'): 'DEL o',
 ('ago', 'ag'): 'DEL o',
 ('ago', 'aga'): 'SUB o => a',
 ('ago', 'agai'): 'ADD i',
 ('ago', 'again'): 'ADD n'}

In [9]:
def parse_solutions(solutions: dict, string1: str, string2: str) -> str:
    route = []
    def helper(solutions: dict, string1: str, string2: str) -> None:
        if not string1 and not string2: return
        operation = solutions[(string1, string2)]
        route.append('{k}: {v}'.format(k=(string1, string2), v=operation))
        if operation.startswith('SUB') or operation == '':
            helper(solutions, string1[:-1], string2[:-1])
        elif operation.startswith('ADD'):
            helper(solutions, string1, string2[:-1])
        elif operation.startswith('DEL'):
            helper(solutions, string1[:-1], string2)
    helper(solutions, string1, string2)
    return '->'.join(route)

In [10]:
parse_solutions(solutions, string1, string2)

"('ago', 'again'): ADD n->('ago', 'agai'): ADD i->('ago', 'aga'): SUB o => a->('ag', 'ag'): ->('a', 'a'): "

In [ ]:
### 在阅读SICP, Structure and Interpretation of Computer Programming 和Fluent Python 两本书，同时参加相应的网上课程